In [1]:
import pandas as pd 
#from numba import jit


tables = ["ethereum.aave.ez_deposits", 
          "ethereum.aave.ez_liquidations", 
          "ethereum.aave.ez_repayments", 
          "ethereum.aave.ez_borrows", 
          "ethereum.aave.ez_withdraws"]

dfDeposits = pd.read_csv(tables[0].replace(".", "_") + ".csv")
dfliquidations = pd.read_csv(tables[1].replace(".", "_") + ".csv")
dfRepayments = pd.read_csv(tables[2].replace(".", "_") + ".csv")
dfBorrows = pd.read_csv(tables[3].replace(".", "_") + ".csv")
dfWithdrawals = pd.read_csv(tables[4].replace(".", "_") + ".csv")

# Need to clean up data that is pulled from DB 
dfDeposits['Time'] = dfDeposits['block_timestamp'].astype('datetime64[ns]')
dfliquidations['Time'] = dfliquidations['block_timestamp'].astype('datetime64[ns]')
dfRepayments['Time'] = dfRepayments['block_timestamp'].astype('datetime64[ns]')
dfBorrows['Time'] = dfBorrows['block_timestamp'].astype('datetime64[ns]')
dfWithdrawals['Time'] = dfWithdrawals['block_timestamp'].astype('datetime64[ns]')


/tmp/ipykernel_21502/2919981745.py:12: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30) have mixed types. Specify dtype option on import or set low_memory=False.
  dfliquidations = pd.read_csv(tables[1].replace(".", "_") + ".csv")
/tmp/ipykernel_21502/2919981745.py:13: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30,33) have mixed types. Specify dtype option on import or set low_memory=False.
  dfRepayments = pd.read_csv(tables[2].replace(".", "_") + ".csv")
/tmp/ipykernel_21502/2919981745.py:14: DtypeWarning: Columns (5,6,9,10,13,17,18,21,22,25,26,28,30,33,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  dfBorrows = pd.read_csv(tables[3].replace(".", "_") + ".csv")


In [2]:
assets = dfDeposits['symbol'].unique()

In [51]:
assets

array(['WETH', 'DAI', 'AAVE', 'SNX', 'REN', 'TUSD', 'UNI', 'USDT', 'MKR',
       'LINK', 'YFI', 'WBTC', 'ZRX', 'USDC', 'ENJ', 'KNC', 'sUSD', 'MANA',
       'BUSD', 'BAT', 'CRV', 'GUSD', 'BAL', 'xSUSHI', 'renFIL', 'RAI',
       'AMPL', 'USDP', 'DPI', 'FRAX', 'FEI', 'stETH', 'ENS', 'UST', 'CVX',
       '1INCH', 'LUSD'], dtype=object)

In [48]:
from datetime import timedelta

for m in range(6,13):
    print(pd.to_datetime(f"2022-{m}-1"))

2022-06-01 00:00:00
2022-07-01 00:00:00
2022-08-01 00:00:00
2022-09-01 00:00:00
2022-10-01 00:00:00
2022-11-01 00:00:00
2022-12-01 00:00:00


In [66]:
month = 7 

collateral = 'USDC'
date = pd.to_datetime(f"2022-{month}-1")
depositsReduced = dfDeposits[(dfDeposits.Time < date) & (dfDeposits.symbol == collateral)]


minAmount = 20000

users = depositsReduced[depositsReduced.supplied_usd > minAmount]['depositor_address'].unique()
df = pd.DataFrame({'user' : users})

dfDepositsClean = dfDeposits[(dfDeposits.depositor_address.isin(users) & (dfDeposits.Time < date))]
dfliquidationsClean = dfliquidations[(dfliquidations.borrower.isin(users) & (dfliquidations.Time < date))]
dfRepaymentsClean = dfRepayments[(dfRepayments.depositor_address.isin(users) & (dfRepayments.Time < date))]
dfBorrowsClean = dfBorrows[(dfBorrows.depositor_address.isin(users) & (dfBorrows.Time < date))]
dfWithdrawalsClean = dfWithdrawals[(dfWithdrawals.depositor_address.isin(users) & (dfWithdrawals.Time < date))]


In [67]:
dfliquidationsClean.head()

,Unnamed: 0,tx_hash,block_number,block_timestamp,event_index,aave_market,aave_token,issued_tokens,supplied_usd,depositor_address,...,debt_aave_token,debt_to_cover_amount,debt_to_cover_amount_usd,liquidator,borrower,collateral_token_price,collateral_token_symbol,debt_token_price,debt_token_symbol,Time
406605,9,0xe4cd7ff0d68ece60648049e92d693f668c26bf240b14...,11502586,2020-12-22 09:57:27.000,214,NaN,NaN,NaN,NaN,NaN,...,0x5165d24277cd063f5ac44efd447b27025e888f37,1.470956,34462.046452,0x0ebe59b1dfdc730cc087cbc5c7bd351a1ce880bc,0x0afde4bfb50f3b908440c7e04ba7d45aa9f5ce7f,1.003457,USDC,23428.333325,YFI,2020-12-22 09:57:27
406606,10,0x368c7f16b248e7eb0f5df6b8eea11d25c1ddd5905086...,11502178,2020-12-22 08:25:48.000,175,NaN,NaN,NaN,NaN,NaN,...,0xa361718326c15715591c299427c62086f69923d9,129404.769186,128993.968618,0x0ebe59b1dfdc730cc087cbc5c7bd351a1ce880bc,0x5382859146010ac16e142b6708f8e178e77f66bc,22257.897982,WBTC,0.996825,BUSD,2020-12-22 08:25:48
406607,11,0x19504b1d2755179e3646c7de1ec9fded4761507209c4...,11512504,2020-12-23 22:18:45.000,48,NaN,NaN,NaN,NaN,NaN,...,0xbcca60bb61934080951369a648fb03df4f96263c,166270.567319,165281.410970,0xfdee74e6148dd4953d54314f2b37e168b57969ff,0x59655511bc501dec7193f2928aad4dbeea1afbd7,10.328588,LINK,0.994051,USDC,2020-12-23 22:18:45
406611,15,0xb04d1e7d847579411b0c0d3c332cdd91f0524aaba673...,11513561,2020-12-24 02:22:37.000,74,NaN,NaN,NaN,NaN,NaN,...,0x3ed3b47dd13ec9a98b44e6204a523e766b225811,7272.607696,7252.036310,0x0ebe59b1dfdc730cc087cbc5c7bd351a1ce880bc,0x8018b586e6c6ceae979d83fe2b938de88b454793,10.721948,LINK,0.997171,USDT,2020-12-24 02:22:37
406616,20,0xc342281fe46ea37feed0aab3ab4b57153dd078f87b9e...,11537672,2020-12-27 19:17:46.000,31,NaN,NaN,NaN,NaN,NaN,...,0x030ba81f1c18d280636f32af80b9aad02cf0854e,17.788193,12395.345230,0x2ca158422b940c6722640ac7fa726e8201cccd33,0xb46660091553d40433e939eb94fabf26420bf2d5,1.002648,DAI,696.829937,WETH,2020-12-27 19:17:46


In [68]:
assetList = ['USDC', 'WETH', 'USDT', 'DAI']

for asset in assetList : 

    assetDeposits = dfDepositsClean[dfDepositsClean.symbol == asset]
    assetBorrows = dfBorrowsClean[dfBorrowsClean.symbol == asset]
    assetWithdraws = dfWithdrawalsClean[dfWithdrawalsClean.symbol == asset]
    assetRepayments = dfRepaymentsClean[dfRepaymentsClean.symbol == asset]
    assetLiquidations = dfliquidationsClean[dfliquidationsClean.collateral_token_symbol == asset]

    def find_first_deposit(row):    
        return assetDeposits[assetDeposits.depositor_address == row['user']]['Time'].min()

    def find_first_borrow(row):    
        return assetBorrows[assetBorrows.borrower_address == row['user']]['Time'].min()

    def find_total_deposits(row):    
        return assetDeposits[assetDeposits.depositor_address == row['user']]['issued_tokens'].sum()

    def find_total_borrows(row):    
        return assetBorrows[assetBorrows.borrower_address == row['user']]['borrowed_tokens'].sum()

    def find_total_withdrawals(row):    
        return assetWithdraws[assetWithdraws.depositor_address == row['user']]['withdrawn_tokens'].sum()

    def find_total_liquidations(row):    
        return assetBorrows[assetBorrows.borrower_address == row['user']]['liquidated_amount'].sum()

    df['firstDeposit' + asset] = df.apply(find_first_deposit,axis =1)
    df['firstBorrow' + asset] = df.apply(find_first_borrow,axis =1)  

    df['daysSinceDeposit' + asset] = date - df['firstDeposit' + asset]
    df['daysSinceBorrow' + asset] = date - df['firstDeposit' + asset]     

    df['totalDeposits' + asset] = df.apply(find_total_deposits,axis =1)
    df['totalBorrowed' + asset] = df.apply(find_total_borrows,axis =1)
    df['totalWithdraws' + asset] = df.apply(find_total_withdrawals,axis =1)
    df['totalLiquidated' + asset] = df.apply(find_total_liquidations,axis =1)




In [92]:
df[collateral + 'Balance'] = df['totalDeposits' + collateral] - df['totalWithdraws' + collateral] - df['totalLiquidated' + collateral]

# how far in the future do we want to predict 
futureDays = 90
futureDate = date + timedelta(days = futureDays)

dfWithdrawFuture = dfWithdrawals[(dfWithdrawals.depositor_address.isin(users) & ((dfWithdrawals.Time >= date) & (dfWithdrawals.Time  < futureDate)))]
dfWithdrawFuture = dfWithdrawFuture[dfWithdrawFuture.symbol == collateral]

df[collateral + 'Withdraw'] = df.user.isin(dfWithdrawFuture.depositor_address)

In [93]:
df[collateral + 'Withdraw'].describe()

count      8291
unique        2
top       False
freq       7616
Name: USDCWithdraw, dtype: object

In [95]:
type(df['daysSinceDeposit' + asset].iloc[0])

pandas._libs.tslibs.timedeltas.Timedelta

In [96]:
for asset in assetList : 
    df['daysSinceDeposit_' + asset] = df['daysSinceDeposit' + asset] / timedelta(hours = 24)
    df['daysSinceBorrow_' + asset] = df['daysSinceBorrow' + asset] / timedelta(hours = 24)

In [97]:
from fastai.tabular.all import *

# Here we want to predict if a users will withdraw or not in the next X days based on their historical interactions with AAVE 

cat_names = []
cont_names = []
for asset in assetList : 
    cont_names.append('daysSinceDeposit_' + asset)
    cont_names.append('daysSinceBorrow_' + asset)
    cont_names.append('totalDeposits' + asset)
    cont_names.append('totalBorrowed' + asset)
    cont_names.append('totalWithdraws' + asset)
    cont_names.append('totalLiquidated' + asset)

splits = RandomSplitter(valid_pct=0.2)(range_of(df))

procs = [Categorify, FillMissing, Normalize]
dls = TabularDataLoaders.from_df(df, '', procs=procs, cat_names=cat_names, cont_names=cont_names, 
                                 y_names=collateral + 'Withdraw', splits=splits, bs=64)
learn = tabular_learner(dls, metrics=accuracy)



In [98]:
learn.fit_one_cycle(5)


epoch,train_loss,valid_loss,accuracy,time
0,0.593163,0.550359,0.841978,00:00
1,0.387193,0.317595,0.907720,00:00
2,0.288442,0.259789,0.916164,00:00
3,0.247378,0.270546,0.914355,00:00
4,0.234956,0.257199,0.916767,00:00
